# Projeto 1 - Ciência dos Dados

Nome: Luca Coutinho Melão

Nome: Thiago de Donato Youssef

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [52]:
!pip install emoji

In [53]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from emoji import UNICODE_EMOJI

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [54]:
import os

filename = 'PS5.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo PS5.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [55]:
train = pd.read_excel(filename)
train.head(20)

,Treinamento,Relevância
0,a sony anunciando o preço do ps5 hoje\nhttps:/...,0
1,vou prar as minhas atividades de desenvolvimen...,0
2,alguma live do evento do ps5 com tradução em p...,1
3,"@thecure70s pô pior que n sei, mas vai ser dep...",1
4,@fiaspo olha... e se rolar gameplay de tlou 2 ...,1
5,espero que o ps5 saia por 10 reais e os jogos ...,0
6,"hoje é dia, né?\nquest, ps5, hero9... \ncrysis...",1
7,#ps5 daqui a pouco saberemos o preço oficial 😬...,0
8,essa dlc de spider man tá linda #ps5,1
9,"n tá com cara de ser o ffxvi não, tá meio basi...",0


In [56]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test

,Teste,Relevância
0,daqui a meia hora tem novidades do #ps5! ansio...,1
1,"@smashrs to plugado, ansioso pela decepção em ...",1
2,"eu: n vou ver o ps5 showcase, tenho muita cois...",0
3,será que vai rolar o preço do ps5? mesmo saben...,1
4,rt @ojovemgamer: quando a sony divulgar o preç...,0
...,...,...
348,"""preço e data o que vier é lucro"" kkkkkkk\n#ps...",1
349,bora ver o preço do ps5 juntos? https://t.co/...,0
350,rt @thiagomoreiracs: a única coisa que eu espe...,1
351,"se tiver god of war ""2"" nesse evento do ps5, e...",1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O PS5 É O NOVO CONSOLE DE JOGOS DA SONY, COM SEU LANÇAMENTO AO FINAL DE 2020.

RELEVANTE

Tweets que expressam o sentimento do usuário com relação ao console, remetem geralmente às expectativas sobre os preços, jogos mais aguardados, comparações com a concorrência.

IRRELEVANTE

Tweets que não estão relacionados ao produto, ou que não agregam nada à análise de sentimentos sobre ele. Tais como: anúncios de lives e comentários sem significado.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [69]:
# Separa emojis dos outros caracteres
def separa_emoji(text):
    return ''.join(' ' + char if char in UNICODE_EMOJI else char for char in text).strip()

# https://medium.com/analytics-vidhya/simplify-your-dataset-cleaning-with-pandas-75951b23568e
def clean_df(dados,base):
    """
        Função que limpa caracteres especiais e troca por espaços
    """
    spec_chars = ["!","˜","rt","\n",'"',"#","%","&","'","(",")",
                  "*","+",",","-",".","/",":",";","<",
                  "=",">","?","[","\\","]","^","_",
                  "`","{","|","}","~","–","https","@"]
    for char in spec_chars:
        dados[base] = dados[base].str.replace(char, ' ')

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

# Dicionarios que guardam quantas vezes uma palavra apareceu na base de dados de acordo com a sua relevância

a = 1
b = 1
    
for tweet in train.loc[train.Relevância==0, 'Treinamento']:
    tweet = cleanup(tweet)
    words = tweet.split()
    for word in words:
        if word not in dc_0.keys():
            dc_0[word] = 1
        else:
            dc_0[word] += 1
                
for tweet in train.loc[train.Relevância==1, 'Treinamento']:
    tweet = cleanup(tweet)
    words = tweet.split()
    for word in words:
        if word not in dc_1.keys():
            dc_1[word] = 1
        else:
            dc_1[word] += 1
    
for i in dc_0.values():
    a += i
for i in dc_1.values():
    b += i
        
den = len(dc_1)
repetido = False
    
for i in dc_0.keys():
    for x in dc_1.keys():
        if i == x:
            repetido = True
    if repetido == False:
        den += 1
            
den_0 = a+den
den_1 = b+den
            
dic = [dc_0,dc_1,den_0,den_1]

# Função que calcula a probabilidade de um tweet não ser relevante
def calcula_prob_0(tweet,dic):
    tweet = cleanup(tweet)
    dc_0 = dic[0]
    den = dic[2]
    prob = 1
    words = tweet.split()
    for word in words:
        if word in dc_0.keys():
            prob *= ((dc_0[word]+1)/den)
        else:
            prob *= ((1)/den)
    return prob

# Função que calcula a probabilidade de um tweet ser relevante
def calcula_prob_1(tweet,dic):
    tweet = cleanup(tweet)
    dc_1 = dic[1]
    den = dic[3]
    prob = 1
    words = tweet.split()
    for word in words:
        if word in dc_1.keys():
            prob *= ((dc_1[word]+1)/den)
        else:
            prob *= ((1)/den)
    return prob

# Função que classifica os tweets
def classificador(dados,base,dic):
    clean_df(dados,base)
    for tweet in dados[base]:
        P_T_dado_R = calcula_prob_1(tweet,dic)
        P_T_dado_Rc = calcula_prob_0(tweet,dic)
        if (P_T_dado_R > P_T_dado_Rc):
            dados.loc[(dados[base]==tweet),'Relevância_Teste'] = 1
        else:
            dados.loc[(dados[base]==tweet),'Relevância_Teste'] = 0
        separa_emoji(tweet)
        

# Classificando base de testes
classificador(test,'Teste',dic)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [76]:
# Tabela que demonstra o teste do classificador com a base de Testes
tab = pd.crosstab(test.Relevância, test.Relevância_Teste)
tab

Relevância_Teste,0.0,1.0
Relevância,,
0,96,72
1,49,136


___
### Concluindo

In [87]:
tab[0][0]
#len(test)

96

In [85]:
# Mostrar as porcentagens de erros/acertos e justificar

#positivos_treinamento = dados.loc[test['Relevância'] == '1','Relevância']
#negativos_treinamento = dados.loc[test['Relevância'] == '0','Relevância']
#positivos_teste = dados.loc[test['Relevância_Teste'] == '1','Relevância']
#negativos_teste = dados.loc[test['Relevância_Teste'] == '1','Relevância']

VP = tab[1][1]
FP = tab[1][0]
VN = tab[0][0]
FN = tab[0][1]
total = VP+FP+VN+FN

print(f'Porcentagem de verdadeiros positivos: {VP/total}')
print(f'Porcentagem de falsos positivos: {FP/total}')
print(f'Porcentagem de verdadeiros negativos: {VN/total}')
print(f'Porcentagem de falsos negativos: {FN/total}')

Porcentagem de verdadeiros positivos: 0.38526912181303113
Porcentagem de falsos positivos: 0.20396600566572237
Porcentagem de verdadeiros negativos: 0.2719546742209632
Porcentagem de falsos negativos: 0.1388101983002833


Por que não posso usar o próprio classificador para gerar mais amostras de treinamento?

Ao utilizar o próprio classificador para gerar mais amostras de treinamento, os eventuais erros cometidos por ele serão armazenados e propagados adiante. Dessa forma, novas amostras de treinamento geradas por ele mesmo carregariam erros consigo e diminuiriam a eficácia do classificador.

Diferentes cenários para Naive Bayes fora do contexto do projeto

Primeiro cenário: Caixa de spam do email
A partir de uma análise de palavras-chave e assuntos recorrentes, o classificador Naive Bayes é capaz de categorizar emails que contém promoções, ofertas, anúncios, etc, como "spam".

Segundo cenário: Correntes de fake news em redes sociais
As correntes de redes sociais que contém fake news usualmente seguem uma tendência, elas apresentam textos em caps-lock, não apresentam fontes/links confiáveis, apelam para o discurso emotivo, e pedem para que sejam imediatamente compartilhadas. A partir da identificação de tais elementos no texto, o classificador Naive Bayes é capaz de sinalizar quando a mensagem contém fake news, ou pelo menos apontar quando há alta probabilidade disso.

Terceiro cenário: Algoritmos de recomendação
Em aplicativos como Netflix ou Spotify é comum ver uma seção de "recomendados para você" ou "acho que você vai gostar". Tendo o conteúdo consumido pelo usuário em um período recente como base de treinamento, o classificador Naive Bayes pode ser utilizado para recomendar novos títulos que possivelmente irão agradar o usuário. Para isso, ele levará em conta o gênero do filme/música, ano de lançamento, palavras das "lyrics", etc.

Quarto cenário: Classificação de documentos
No meio corporativo e jurídico é muito comum a necessidade de lidar com diversos documentos, tais como procurações, petições, contratos sociais, acordos de confidencialidade, etc. Sendo assim, a partir de palavras-chave presentes em cada tipo, o classificador Naive Bayes é capaz de categorizar os documentos em seus respectivos grupos.

Quinto cenário: Categorização de artigos 
Ao acessar uma newsletter ou ler as notícias, é muito comum que os artigos estejam divididos de acordo com suas categorias como esportes, tecnologia, economia, entre outros. O classificador Naive Bayes pode fazer essa divisão também levando em conta palavras-chave presentes em cada categoria.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**